In [72]:
import pandas as pd
import numpy as np
from scipy import io
import plotly.express as px
import math
import re
import random as rnd

In [3]:
sectors = [
    'Mining', 'Util', 'Const', 'Wood', 'Minerals', 'Primary Metals', 'Fabricated Metals',
    'Machinery', 'Computers', 'Electrical', 'Vehicles', 'Transport', 'Furniture',
    'Misc Mfg', 'Food Mfg', 'Textile', 'Apparel', 'Paper', 'Printing', 'Petroleum', 
    'Chemical', 'Plastics', 'Wholesale Trade', 'Retail Trade', 'Transit/Warehouse', 
    'Info', 'Finance/Insurance', 'Real Estate', 'Prof/Tech', 'Mgmt', 'Admin', 'Educ', 
    'Health', 'Arts', 'Accom', 'Food Services', 'Other Services'
]


In [4]:
invnet = io.loadmat('./investmentNetwork/Replication Packet/Investment Network Construction/invmatdat_37.mat')

In [5]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))
avg_net.columns = sectors
avg_net.index = sectors

The investment network was constructed based on sector-level asset investments and production using the below equation:

$$ I_{ijt} = \sum_{a=1}^{A}\omega_{iat} I_{ajt}^{exp}$$

Where $I$ is the (37x37x72) investment matrix for 1947-2018. $I_{ajt}^{exp}$ is the expenditure by sector $j$ on asset $a$ in year $t$. $\omega_{iat}$ is the fraction of asset $a$ produced by sector $i$ in year $t$. The network below is the mean (37x37) matrix across time. 

The original network relies on a number of assumptions that largely contribute to the investment hubs it identified. These assumptioins are mostly due to the lack of data on sector-level asset production.

1. Construction sector produces all non-mining structures
2. Prof/Technical Services produce all custom software and R&D
3. Artistic originals are only produced by info/comm or arts
4. Info/comm produce all pre-packaged software but not delivery
5. Wood Manufacturing, insurance/finance, and Prof/Tech services produce residential structures but not non-residential structures

In addition to these theoretic assumptions, much of the data pre-1997 is estimated bluntly by taking the average production/expenditure ratios post-1997 or interpolating with data from 1987 and 1992.

Moreover, the asset expenditure data was flagged as unreliable by BEA and it was specifically said they're "more likely to be based on judgemental trends, on trends in the higher level aggregate, or on less reliable source data." On the other hand, the asset production data is mostly just for "equipment" and the allocations are based on the assumptions above.

Are there asset classes baked into the macro model? If so, that would be a natural set of assets to base an investment network off of

In [6]:
fig = px.imshow(avg_net)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

### Simple Version of Investment Network

In this network, I aggregate the sectors into 18 that map more cleanly between BEA codes and the ISIC codes and attempt to construct an investment network using the same methods as VA. The loss is the granularity of the data into 37+ sectors and I will only use 1997+ data.

In [7]:
pd.read_excel('assetInvestments.xlsx', sheet_name='110C', header=5)

,Asset Codes,NIPA Asset Types,1901,1902,1903,1904,1905,1906,1907,1908,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EQUIPMENT,TOTAL EQUIPMENT,83.0,116.0,90.0,94.0,98.0,121.0,162.0,142.0,...,46746.0,52774.0,61585.0,44154.0,35166.0,39459.0,39021.0,43800.0,45455.0,48770.0
2,EP1A,Mainframes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EP1B,PCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,5.0,5.0,3.0,3.0,3.0,3.0,3.0,4.0,5.0
4,EP1C,DASDs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AE10,Theatrical movies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,AE20,Long-lived television programs ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,AE30,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,AE40,Music,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Attempt at Mapping BEA -> NAICS -> ISIC/NACE

One of the issues with the mapping between NAICS and ISIC is that both the Investment Network paper and the macro model use intermediate aggregates of the coding scheme while the mapping is on the lowest-level. I haven't found any mappings that map this intermediate level so I have to map from NAICS-mid -> NAICS-low -> ISIC-low -> ISIC-mid. Which is a many-to-many mapping. 

The approach I'm taking above maps the highest-level aggregate sectors into 18 relatively clean-to-match sectors and reconstructs an investment matrix for these using VW's approach.

In [8]:
# compares to "Replication Packet/Converting SIC to NAICS/SIC_NAICS_BEA_allsec.do"

NAICS_groups = {
    'Agriculture': [111,112,113,114,115], # excluded
    'Mining': [212],
    'Oil/Gas': [211], # excluded
    'Mining Support': ['212'], # excluded
    'Util': [22], # 221 doesn't exist in NAICS codes in asset expenditures but in VW mapping
    'Const': [23],
    'Wood': [321],
    'Minerals': [327],
    'Primary Metals': [331],
    'Fabricated Metals': [332],
    'Machinery': [333],
    'Computers': [334],
    'Electrical': [335],
    'Vehicles': [3361,3362,3363],
    'Transport': [3364,3365,3366,3367,3368,3369],
    'Furniture': [337],
    'Misc Mfg': [339],
    'Food Mfg': [311,312],
    'Textile': [313,314],
    'Apparel': [315,316],
    'Paper': [322],
    'Printing': [323],
    'Petroleum': [324],
    'Chemical': [325],
    'Plastics': [326],
    'Wholesale Trade': [42],
    'Retail Trade': [44,45],
    'Transit/Warehouse': [48,49],
    'Info': [51],
    'Finance/Insurance': [52],
    'Real Estate': [531],
    'Rental': [532,533], # excluded
    'Prof/Tech': [54],
    'Mgmt': [55],
    'Admin': [561,562], # just 56 in the VW mapping
    'Educ': [61],
    'Health': [62],
    'Arts': [71],
    'Accom': [721],
    'Food Services': [722],
    'Other Services': [81]
}

In [9]:
naics_isic_df = pd.read_csv('NAICS_ISIC.csv', dtype={'NAICS2012Code': str, 'ISIC4Code': str})

naics_isic_map = {}
for i, row in naics_isic_df.iterrows():
    naics_isic_map[str(row[0])] = row[2]
    

In [10]:
ISIC_groups = {}
for key, ids in NAICS_groups.items():
    ISIC_groups[key] = []
    for i in ids:
        for naics_code, isic_code in naics_isic_map.items():
            id_len = len(str(i))
            if not naics_code is None and naics_code[0:id_len] == str(i):
                ISIC_groups[key].append(str(isic_code))
    ISIC_groups[key] = list(set(ISIC_groups[key]))

In [11]:
isic_nace_df = pd.read_csv('ISIC_NACE.csv', dtype={'ISIC4code': str, 'NACE2code': str})

isic_nace_map = {}
for i, row in isic_nace_df.iterrows():
    isic_nace_map[str(row[0])] = row[2]

In [12]:
NACE2_groups = {}
for key, ids in ISIC_groups.items():
    NACE2_groups[key] = []
    for i in ids:
        for isic_code, nace_code in isic_nace_map.items():
            id_len = len(str(i))
            if not isic_code is None and isic_code[0:id_len] == str(i):
                NACE2_groups[key].append(str(nace_code))
    NACE2_groups[key] = np.sort(list(set(NACE2_groups[key])))

In [13]:
# mining == mining support (08), fabricated metals == furniture (25)
NACE2_groups_agg = {key: pd.Series([x[0:2] for x in val]).value_counts().index for key, val in NACE2_groups.items()} 
# mining == mining support (08), minerals == petroleum (23)
ISIC_groups_agg = {key: pd.Series([x[0:2] for x in val]).value_counts().index for key, val in ISIC_groups.items()}

In [68]:
sea_df = pd.read_csv('sample_data/Socio_Economic_Accounts.csv')

our_map = {}
for i, row in sea_df.iterrows():
    our_map[row[2]] = row[3]

In [62]:
def clean_isic(x):
    ids = re.findall('\d{2}', x)
    if len(ids) == 0:
        return [x]
    elif len(ids) == 1:
        return [int(x[1:])]
    else:
        int_ids = [int(id) for id in ids]
        min = np.min(int_ids)
        max = np.max(int_ids)
        return list(np.arange(min, max))
    
clean_our_map = {}
for key in our_map.keys():
    clean_our_map[key] = clean_isic(our_map[key])

In [70]:
isic_ours_map = {}
for isic in ISIC_groups_agg.items():
    isic_ours_map[isic[0]] = []
    for ours in clean_our_map.items():
        if len(set(ours[1]).intersection(set([int(i) for i in isic[1]]))) > 0:
            isic_ours_map[isic[0]].append(our_map[ours[0]])

In [71]:
isic_ours_map_manual_edit = {
    'Agriculture': ['A01', 'A02', 'A03', 'C10-C12', 'C16'],
    'Mining': ['B'],
    'Oil/Gas': [],
    'Mining Support': ['B'],
    'Util': ['D35', 'E37-E39', 'H49'],
    'Const': ['F'],
    'Wood': ['C16', 'C31_C32'],
    'Minerals': ['C23'],
    'Primary Metals': ['C24', 'C25', 'C27', 'C28', 'C30'],
    'Fabricated Metals': ['C24', 'C25', 'C28', 'C29', 'C30'],
    'Machinery': ['C25', 'C26', 'C28', 'C30'],
    'Computers': ['C18', 'C26', 'C30'],
    'Electrical': ['C27'],
    'Vehicles': ['C25', 'C29', 'C30'],
    'Transport': ['C30', 'C33'],
    'Furniture': ['C25', 'C31_C32'],
    'Misc Mfg': ['C27', 'C28'],
    'Food Mfg': ['C10-C12', 'C20', 'D35'],
    'Textile': ['C13-C15', 'C22'],
    'Apparel': ['C13-C15', 'C22'],
    'Paper': ['C17', 'C23', 'C25'],
    'Printing': ['C18'],
    'Petroleum': ['C19', 'C23'],
    'Chemical': ['C20', 'C21', 'C22', 'C28'],
    'Plastics': ['C22', 'C27', 'C31_C32'],
    'Wholesale Trade': ['G45', 'G46'],
    'Retail Trade': ['G45', 'G47'],
    'Transit/Warehouse': ['H49', 'H50', 'H51', 'H52', 'H53'],
    'Info': ['J58', 'J59_J60', 'J61', 'M74_M75'],
    'Finance/Insurance': ['K64', 'K65', 'K66', 'L68'],
    'Real Estate': ['L68'],
    'Rental': [],
    'Prof/Tech': ['J62_J63', 'M69_M70', 'M71', 'M72', 'M73', 'M74_M75'],
    'Mgmt': ['K64'],
    'Admin': ['E37-E39', 'O84'],
    'Educ': ['P85'],
    'Health': ['Q'],
    'Arts': ['M74_M75'],
    'Accom': ['I'],
    'Food Services': ['I'],
    'Other Services': ['C33', 'G45', 'M71', 'M74_M75']
 }

In [81]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))
selection = [rnd.choice(isic_ours_map_manual_edit[s]) for s in sectors]
avg_net.columns = selection
avg_net.index = selection

In [82]:
fig = px.imshow(avg_net)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

In [79]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))
avg_net.columns = sectors
avg_net.index = sectors

In [80]:
fig = px.imshow(avg_net)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()